In [ ]:
import torch
from training.model import SIREN
import numpy as np
from render.meshing import create_mesh
import open3d as o3d
import json
import meshplot as mp
import matplotlib.cm as cm

In [ ]:
EXPERIMENT = 'juguete'
LOSS = 'dgni'
BRANCH = 0

model = SIREN(
        n_in_features=4,
        n_out_features=1,
        hidden_layer_config=[256, 256, 256, 256],
        w0=60,
        ww=None
)
model.load_state_dict( torch.load(f'../results/{EXPERIMENT}/test_{EXPERIMENT}_{LOSS}/models/model_best.pth'))

meshes_pred = []
meshes_gt = []
with open(f'../results/{EXPERIMENT}/{EXPERIMENT}.json') as jsonFile:
    skel = json.load(jsonFile)
    branch = skel['branches'][BRANCH]

    for joint in branch['joints']:
        vertices, faces, normals, values = create_mesh(
            model,
            distance = torch.from_numpy(np.array(joint['distance']).astype(np.float32)),
            N=128
        )
        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(vertices), o3d.utility.Vector3iVector(faces) )
        mesh.transform( np.array(joint['transformation']).astype(np.float32) )
        meshes_pred.append(mesh)

        mesh = o3d.geometry.TriangleMesh( o3d.utility.Vector3dVector(np.array(joint['vertices']).astype(np.float32)), o3d.utility.Vector3iVector(np.array(joint['triangles']).astype(np.float32)) )
        mesh.transform( np.array(joint['transformation']).astype(np.float32) )
        meshes_gt.append(mesh)


In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_pred):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_pred)))[0:3])
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_pred)))[0:3] )

In [ ]:

p = None
cmap = cm.get_cmap('flag')
for i, mesh in enumerate(meshes_gt):
    if p is None:
        p = mp.plot( np.asarray(mesh.vertices), np.asarray(mesh.triangles), return_plot = True, c=np.array(cmap((i)/len(meshes_gt)))[0:3])
    else:
        p.add_mesh( np.asarray(mesh.vertices), np.asarray(mesh.triangles), c=np.array(cmap((i)/len(meshes_gt)))[0:3] )